В этом блокноте мы рассмотрим некоторые практические аспекты использования больших языковых моделей, а именно доступность и подбор промтов.

# Использование LLM в Google Colab

Существует несколько способов использовать большие языковые модели в среде Google Colab. Для всех них необходимы ключи доступа, которые можно хранить в разделе "Секреты".

In [ ]:
from google.colab import userdata

## Yandex Cloud

[Доступны](https://yandex.cloud/ru/docs/foundation-models/concepts/yandexgpt/models) модели серии YandexGPT и LLama.

Плюсы: быстрая работа.

Минусы: платное использование по завершении приветственного гранта в 4000 рублей.

Цена за 1000 токенов:
- YandexGPT Lite (`yandexgpt-lite`) — 0,20 ₽
- YandexGPT Pro (`yandexgpt`) — 1,20 ₽
- Llama 8b (`llama-lite`) — 0,20 ₽
- Llama 70b (`llama`) — 1,20 ₽

Для доступа необходимы API-ключ и идентификатор модели.

In [ ]:
url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
yandex_gpt = userdata.get("yandex_gpt")
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Api-Key {yandex_gpt}" # ваш секретный ключ
}

In [ ]:
%%time
import requests
import json

prompt = {
    "modelUri": "gpt://b1gu40ajd726f9s9h843/yandexgpt", # идентификатор модели
    "completionOptions": {
        "stream": False, # отключение режима диалога
        "temperature": 1 # степень рандомности генерации
        },
    "messages": [
        {
            "role": "user",
            "text": "Что изучает лингвистика?"
            }
        ]
    }

response = requests.post(url, headers=headers, json=prompt).text
print(response)
text = json.loads(response)["result"]["alternatives"][0]["message"]["text"]
print(text)

In [ ]:
def yandexgpt(content):
  prompt = {
      "modelUri": "gpt://b1gu40ajd726f9s9h843/yandexgpt",
      "completionOptions": {
          "stream": False,
          "temperature": 1
          },
      "messages": [
          {
              "role": "user",
              "text": content
              }
          ]
      }

  response = requests.post(url, headers=headers, json=prompt).text
  text = json.loads(response)["result"]["alternatives"][0]["message"]["text"]
  return text

In [ ]:
%%time
questions = ["Что изучает лингвистика?",
             "Чем отличаются фонетика и фонология?",
             "Почему языки такие разные?"]

for question in questions:
  answer = yandexgpt(question)
  print(f"Вопрос:\n{question}")
  print(f"Ответ:\n{answer}\n")
  print()

## Сбер

[Доступны](https://developers.sber.ru/docs/ru/gigachat/api/tariffs) модели серии GigaChat: Lite (`GigaChat-Lite`), Pro (`GigaChat-Pro`) и Max (`GigaChat-Max`).

Плюсы: быстрая работа.

Минусы: платное использование по завершении приветственного гранта на 50 000 токенов для каждой модели.

Тарификация:

- 5 000 000 токенов GigaChat Lite — 1 000 ₽
- 30 000 000 токенов GigaChat Lite — 5 820 ₽
- 1 000 000 токенов GigaChat Pro — 1 500 ₽
- 5 000 000 токенов GigaChat Pro — 7 275 ₽
- 1 000 000 токенов GigaChat Max — 1 950 ₽
- 4 000 000 токенов GigaChat Max — 7 566 ₽

Для доступа необходимы идентификатор клиента (`RqUID`) и API-ключ (`Authorization`), которые позволяют получить токен доступа (`access_token`). Также нужно указать идентификатор модели.

In [ ]:
import requests
import json

url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

payload={
  'scope': 'GIGACHAT_API_PERS'
}

gc_RqUID = userdata.get("gc_RqUID")
gc_key = userdata.get("gc_key")
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Accept': 'application/json',
  'RqUID': gc_RqUID,
  'Authorization': f'Basic {gc_key}'
}

response = requests.post(url, headers=headers, data=payload, verify=False)
token = response.json().get('access_token')
print(token)

In [ ]:
%%time

url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
headers = {
    'Authorization': 'Bearer ' + token,
    'Content-Type': 'application/json',
    }
payload=json.dumps({
    "model": "GigaChat-Pro",
    "messages": [
        {
            "role": "user",
            "content": "Что изучает лингвистика?"
            }
        ],
    "temperature": 1,
    "stream": False,
    })
response = requests.post(url, headers=headers, data=payload, verify=False)
print(response.text)
text = response.json()['choices'][0]['message']['content']
print(text)

In [ ]:
def gigachat(content):
  url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
  headers = {
      'Authorization': f'Bearer {token}',
      'Content-Type': 'application/json',
      }
  payload=json.dumps({
      "model": "GigaChat-Pro",
      "messages": [
          {
              "role": "user",
              "content": content
              }
          ],
      "temperature": 1,
      "stream": False,
      })
  response = requests.post(url, headers=headers, data=payload, verify=False)
  text = response.json()['choices'][0]['message']['content']
  return text

In [ ]:
%%time
questions = ["Что изучает лингвистика?",
             "Чем отличаются фонетика и фонология?",
             "Почему языки такие разные?"]

for question in questions:
  answer = gigachat(question)
  print(f"Вопрос:\n{question}")
  print(f"Ответ:\n{answer}\n")
  print()

## Mistral AI

[Доступны](https://docs.mistral.ai/getting-started/models/models_overview/) модели серии Mistral:
- мультиязычные LLM
  - Mistral Large (`mistral-large-latest`)
  - Ministral 8B (`ministral-8b-latest`)
  - Ministral 3B	(`ministral-3b-latest`)
- LLM для языков Ближнего Востока и Южной Азии
  - Mistral Saba (`mistral-saba-latest`)
- LLM для кода
  - Codestral (`codestral-latest`)
- мультимодальная LLM
  - Pixtral Large (`pixtral-large-latest`)

Плюсы: бесплатное использование.

Минусы: долгая работа.

Для доступа необходимы API-ключ и идентификатор модели.

In [ ]:
%%time
import requests

mistral = userdata.get("mistral")
headers = {
      'Authorization': f'Bearer {mistral}',
      'Content-Type': 'application/json',
      }
payload = {
    "model": "mistral-large-latest",
    "temperature": 1,
    "stream": False,
    "messages": [
        {
            "role": "user",
            "content": "Что изучает лингвистика?"
            }
        ]
    }
response = requests.post("https://api.mistral.ai/v1/chat/completions", headers=headers, json=payload)
print(response.json())
text = response.json()['choices'][0]['message']['content']
print(text)

In [ ]:
def mistral(content):
  mistral = userdata.get("mistral")
  headers = {
        'Authorization': f'Bearer {mistral}',
        'Content-Type': 'application/json',
        }
  payload = {
      "model": "mistral-large-latest",
      "temperature": 1,
      "stream": False,
      "messages": [
          {
              "role": "user",
              "content": content
              }
          ]
      }
  response = requests.post("https://api.mistral.ai/v1/chat/completions", headers=headers, json=payload)
  text = response.json()['choices'][0]['message']['content']
  return text

In [ ]:
%%time
questions = ["Что изучает лингвистика?",
             "Чем отличаются фонетика и фонология?",
             "Почему языки такие разные?"]

for question in questions:
  answer = mistral(question)
  print(f"Вопрос:\n{question}")
  print(f"Ответ:\n{answer}\n")
  print()

## Together AI

[Доступно](https://api.together.ai/models) достаточно много разных моделей по разным тарифам:
- DeepSeek R1
- Qwen 2.5 Instruct
- Gemma Instruct

Однако есть две бесплатные модели:
- Meta Llama 3.3 70B Instruct Turbo Free (`meta-llama/Llama-3.3-70B-Instruct-Turbo-Free`)
- DeepSeek R1 Distill Llama 70B Free (`deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free`)

Плюсы: возможно бесплатное использование.

Минусы: долгая работа.

Для доступа необходимы API-ключ и идентификатор модели.

In [ ]:
%%time
import os
import openai

together_ai = userdata.get("together_ai")
client = openai.OpenAI(
    api_key=together_ai,
    base_url="https://api.together.xyz/v1",
    )
response = client.chat.completions.create(
    model= "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    temperature=1,
    stream=False,
    messages=[
        {
            "role": "user",
            "content": "Что изучает лингвистика?"
            }
        ]
    )
print(response)
text = response.choices[0].message.content
print(text)

In [ ]:
def llama(content):
  together_ai = userdata.get("together_ai")
  client = openai.OpenAI(
      api_key=together_ai,
      base_url="https://api.together.xyz/v1",
      )
  response = client.chat.completions.create(
      model= "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
      temperature=1,
      stream=False,
      messages=[
          {
              "role": "user",
              "content": content
              }
          ]
      )
  text = response.choices[0].message.content
  return text

In [ ]:
%%time
questions = ["Что изучает лингвистика?",
             "Чем отличаются фонетика и фонология?",
             "Почему языки такие разные?"]

for question in questions:
  answer = llama(question)
  print(f"Вопрос:\n{question}")
  print(f"Ответ:\n{answer}\n")
  print()

## Hugging Face

На Hugging Face [доступно](https://huggingface.co/models?pipeline_tag=text-generation&sort=trending) огромное количество моделей. Модели, дообученные с помощью SFT, обозначены как Instruct.

Возможность использования графического процессора в Google Colab позволяет напрямую импортировать модели до 2 миллиардов параметров.

Плюсы: бесплатное использование.

Минусы: долгая работа, ограничения на размер модели.

Для доступа необходимы GPU и идентификатор модели.

In [ ]:
!pip install transformers -q

In [ ]:
%%time
import torch
from transformers import pipeline

device = ("cuda" if torch.cuda.is_available() else "cpu")

qwen_1_5_b = pipeline("text-generation",
                      model="Qwen/Qwen2.5-1.5B-Instruct",
                      temperature=1,
                      max_new_tokens=1000,
                      device=device)

In [ ]:
%%time
messages = [
    {
        "role": "user",
        "content": "Что изучает лингвистика?"
        },
]
response = qwen_1_5_b(messages)
print(response)
text = response[0]["generated_text"][1]["content"]
print(text)

In [ ]:
def qwen(content):
  messages = [
      {
          "role": "user",
          "content": content
          },
      ]
  response = qwen_1_5_b(messages)
  text = response[0]["generated_text"][1]["content"]
  return text

In [ ]:
%%time

questions = ["Что изучает лингвистика?",
             "Чем отличаются фонетика и фонология?",
             "Почему языки такие разные?"]

for question in questions:
  answer = qwen(question)
  print(f"Вопрос:\n{question}")
  print(f"Ответ:\n{answer}\n")
  print()

Модели большего размера доступны через [InferenceClient](https://huggingface.co/docs/huggingface_hub/v0.16.2/en/package_reference/inference_client#huggingface_hub.InferenceClient).

Плюсы: бесплатное использование.

Минусы: ограничение на 1000 запросов в сутки, долгая работа.

Для доступа необходимы токен и идентификатор модели.

In [ ]:
%%time
from huggingface_hub import InferenceClient

hf_token = userdata.get("hf_token")
model_name = "Qwen/Qwen2.5-72B-Instruct"
client = InferenceClient(model_name, token=hf_token)

output = client.chat.completions.create(
          messages=[
              {
                  "role": "user",
                  "content": "Что изучает лингвистика?"
                  },
              ],
          max_tokens=1000,
          temperature=1)
print(output)
text = output.choices[0].get('message')['content']
print(text)

In [ ]:
def qwen_72(content):
  hf_token = userdata.get("hf_token")
  model_name = "Qwen/Qwen2.5-72B-Instruct"
  client = InferenceClient(model_name, token=hf_token)

  output = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": content
              },
          ],
      max_tokens=1000,
      temperature=1)
  text = output.choices[0].get('message')['content']
  return text

In [ ]:
%%time

for question in questions:
  answer = qwen_72(question)
  print(f"Вопрос:\n{question}")
  print(f"Ответ:\n{answer}\n")
  print()

## GroqCloud

[Доступны](https://console.groq.com/docs/rate-limits) различные модели (сайт открывается через VPN):
- `llama-3.3-70b-versatile`
- `qwen-2.5-32b`
- `deepseek-r1-distill-qwen-32b`
- `deepseek-r1-distill-llama-70b`

Плюсы: бесплатное использование, быстрая работа.

Минусы: ограничение на количество запросов (для каждой модели отличается).

Для доступа необходимы API-ключ и идентификатор модели.

In [ ]:
!pip install groq -q

In [ ]:
%%time
from groq import Groq

groq = userdata.get("groq")
client = Groq(
    api_key=groq,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Что изучает лингвистика?",
        }
    ],
    model="llama-3.3-70b-versatile",
    temperature=1
)

print(chat_completion)
text = chat_completion.choices[0].message.content
print(text)

In [ ]:
def deepseek(content):

  groq = userdata.get("groq")
  client = Groq(
      api_key=groq,
  )

  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": content
              }
          ],
      model="llama-3.3-70b-versatile",
      temperature=1
      )

  text = chat_completion.choices[0].message.content
  return text

In [ ]:
%%time
questions = ["Что изучает лингвистика?",
             "Чем отличаются фонетика и фонология?",
             "Почему языки такие разные?"]

for question in questions:
  answer = deepseek(question)
  print(f"Вопрос:\n{question}")
  print(f"Ответ:\n{answer}\n")
  print()

# Prompt engineering

Затравочное программирование (prompt engineering) — разработка и оптимизация затравок (промптов) для эффективного использования больших языковых моделей.

Рассмотрим различные способы формулировки промптов на примере модуля ChatGroq библиотеки LangChain. Он позволяет использовать модели из GroqCloud.

In [ ]:
!pip install langchain langchain_groq -q

## Базовое использование

Для инициализации модели необходим API-ключ и ее идентификатор.

In [ ]:
from google.colab import userdata
from langchain_groq import ChatGroq

groq = userdata.get("groq")
llm = ChatGroq(
    temperature=1,
    groq_api_key = groq,
    model_name = "llama-3.3-70b-versatile"
)

In [ ]:
basic_prompt = "Что изучает лингвистика?"
print(llm.invoke(basic_prompt).content)

В зависимости от конкретной формулировки промпта ответ может различаться.

In [ ]:
prompts = [
    "Перечисли 4 примера применения искусственного интеллекта в здравоохранении",
    "Объясни, как искусственный интеллект меняет область здравоохранения на 4 конкретных примерах",
    "Ты врач. Опиши 4 способа, с помощью которых искусственный интеллект улучшил твою повседневную работу в больнице"
]

for i, prompt in enumerate(prompts, 1):
  print(f"\nПромпт {i}: ")
  print(prompt)
  print("\nОтвет: ")
  print(llm.invoke(prompt).content)
  print("-"*100)

## Структурированный промпт

Модуль `PromptTemplate` позволяет использовать переменные внутри промпта.

In [ ]:
from langchain.prompts import PromptTemplate

structured_prompt = PromptTemplate(
    input_variables=["topic"],
    template = "Что изучает {topic}?"
)
chain = structured_prompt | llm
input_variables = {"topic": "лингвистика"}
output = chain.invoke(input_variables).content
print(output)

In [ ]:
fact_check_prompt = PromptTemplate(
    input_variables=["statement"],
    template="""Оцени приведенное ниже утверждение на предмет достоверности. Если оно неверно, укажи правильную информацию:
    Утверждение: {statement}
    Оценка:"""
)

input_variables = {"statement": "Столицей Индии является Лондон."}
chain = fact_check_prompt | llm
print(chain.invoke(input_variables).content)

In [ ]:
problem_solving_prompt = PromptTemplate(
    input_variables=["problem"],
    template = """Реши следующую задачу шаг за шагом:
    Задача: {problem}
    Решение:
    1)"""
)

input_variables = {"problem": "Банк предлагает годовую процентную ставку в размере 6%, которая ежегодно увеличивается. \
За последние 5 лет суммы годовых депозитов были следующими: $1000, $1500, $2000, $2500, и $3000 долларов США. \
Рассчитайте общую сумму на счете по истечении 5 лет с учетом ежегодного начисления процентов."}
chain = problem_solving_prompt | llm
print(chain.invoke(input_variables).content)

## Последовательность реплик

Для ведения диалога более чем из 1 реплики используется модуль ConversationChain.

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm = llm,
    verbose = True,
    memory = ConversationBufferMemory()
)

print(conversation.predict(input="Что такое галактика?"))
print(conversation.predict(input="Сколько галактик во Вселенной?"))
print(conversation.predict(input="Как называется галактика, в которой мы живем?"))

Это помогает запоминать предыдущий контекст диалога.

In [ ]:
prompts = [
    "Какой город является столицей Франции?",
    "Каково его население?",
    "Какая самая известная достопримечательность этого города?"
]

for prompt in prompts:
    print(f"Q: {prompt}")
    print(f"A: {llm.invoke(prompt).content}\n")

In [ ]:
conversation = ConversationChain(llm=llm, memory=ConversationBufferMemory())
for prompt in prompts:
    print(f"Q: {prompt}")
    print(f"A: {conversation.predict(input=prompt)}\n")

## Режим zero-shot

При достаточно подробной формулировке инструкции задача может быть решена без каких-либо демонстрационных примеров.

In [ ]:
def create_chain(prompt_template):
  prompt = PromptTemplate.from_template(prompt_template)
  return prompt | llm

In [ ]:
direct_task_prompt = """Классифицируй тональность следующего текста как положительную, отрицательную или нейтральную. \
Не объясняй свои доводы, просто приведи классификацию.

Текст: {text}
Тональность: """

direct_task_chain = create_chain(direct_task_prompt)

texts = [
    "В новом кафе в городе такая уютная атмосфера, и кофе превосходный!",
    "Книга была неплохой, но я бы не сказал, что она выделялась на фоне других, которые я читал.",
    "Опыт онлайн-покупок был разочаровывающим; веб-сайт постоянно зависал."
]

for text in texts:
  result = direct_task_chain.invoke({"text": text}).content
  print(f"Текст: {text}")
  print(f"Тональность: {result}")

Для определенных задач важно детально задать формат ответа.

In [ ]:
format_spec_prompt = """Создай короткую новостную статью по теме "{topic}".
Структурируй свой ответ в следующем формате:

Заголовок: [Броский заголовок для статьи]

Введение: [Краткий вводный абзац, в котором излагаются ключевые моменты]

Основное содержание: [2-3 коротких абзаца с более подробной информацией]

Вывод: [Заключительное предложение или призыв к действию]"""

format_spec_chain = create_chain(format_spec_prompt)

topic = "Прорыв в технологии хранения возобновляемой энергии"
result = format_spec_chain.invoke({"topic": topic}).content
print(result)

Кроме того, можно задать конкретные этапы выполнения задачи.

In [ ]:
multi_step_prompt = """Проанализируй следующий текст на предмет его основного аргумента, подтверждающих доказательств и потенциальных контраргументов.
Проведи свой анализ по следующим этапам:

1. Главный аргумент: Определи и сформулируй основное утверждение или тезис.
2. Подтверждающие доказательства: Перечисли ключевые моменты или доказательства, используемые в поддержку основного аргумента.
3. Возможные контраргументы: Предложите возможные возражения или альтернативные точки зрения на основной аргумент.

Текст: {text}

Анализ:"""

multi_step_chain = create_chain(multi_step_prompt)

text = """В последние годы удаленная работа становится все более популярной, предлагая многочисленные преимущества как сотрудникам, так и работодателям.
Работники пользуются большей гибкостью, сокращают время на дорогу и могут создавать более персонализированную рабочую среду.
Работодатели выигрывают от снижения расходов на офис и доступа к более широкому кадровому резерву.
Однако такие проблемы, как поддержание совместной работы в команде, управление производительностью и обеспечение безопасности данных по-прежнему сохраняются, что делает переход на удаленную работу не лишенным недостатков."""

result = multi_step_chain.invoke({"text": text}).content
print(result)

## Режим few-shot

Добавление примеров помогает добиться лучшего решения задачи.

In [ ]:
def few_shot_sentiment_classification(input_text):
  few_shot_prompt = PromptTemplate(
      input_variables=["input_text"],
      template="""
      Классифицируй тональность как положительную, отрицательную или нейтральную.

      Пример:
      Текст: Мне нравится этот продукт! Он потрясающий.
      Тональность: Положительная
      Текст: Этот фильм был ужасен. Я его возненавидел.
      Тональность: Отрицательная
      Текст: Погода сегодня неплохая.
      Тональность: Нейтральная

      Теперь классифицируй следующее предложение
      Текст: {input_text}
      Тональность:
      """
  )
  chain = few_shot_prompt | llm
  result = chain.invoke(input_text).content
  result = result.strip()

  if ':' in result:
    result = result.split(':')[1].strip()

  return result

test_text = "Я не могу поверить, насколько велик и духовен кедарнатх!"

result = few_shot_sentiment_classification(test_text)
print(f"Текст : {test_text}")
print(f"Тональность: {result}")

In [ ]:
def multi_task_few_shot(input_text, task):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text", "task"],
        template="""
        Выполни указанное задание по данному тексту.

        Примеры:
        Текст: Мне нравится этот продукт! Он потрясающий.
        Задание: тональность
        Результат: положительная

        Текст: Это самый худший опыт, который у меня когда-либо был.
        Задание: тональность
        Результат: отрицательная

        Текст: Bonjour, comment allez-vous?
        Задание: язык
        Результат: французский

        Текст: Guten Tag, wie geht es Ihnen?
        Задание: язык
        Результат: немецкий

        Текст: কেমন আছেন? (Kemon achhen?)
        Задание: язык
        Результат: бенгальский

        Текст: От топота копыт пыль по полю летит.
        Задание: подсчет слов
        Результат: 7

        Теперь выполни следующее задание:
        Текст: {input_text}
        Задание: {task}
        Результат:
        """
    )

    chain = few_shot_prompt | llm
    return chain.invoke({"input_text": input_text, "task": task}).content

print(multi_task_few_shot("Я не могу поверить, насколько это здорово!", "тональность"))
print(multi_task_few_shot("Guten Tag, wie geht es Ihnen?", "язык"))
print(multi_task_few_shot("কেমন আছেন?", "язык"))
print(multi_task_few_shot("Бык тупогуб, тупогубенький бычок, у быка бела губа была тупа.", "подсчет слов"))

In [ ]:
def in_context_learning(task_description, examples, input_text):
    example_text = "".join([f"Ввод: {e['input']}\nВывод: {e['output']}\n\n" for e in examples])

    in_context_prompt = PromptTemplate(
        input_variables=["task_description", "examples", "input_text"],
        template="""
        Задание: {task_description}

        Примеры:
        {examples}

        Теперь выполни задание со следующими входными данными:
        Ввод: {input_text}
        Вывод:
        """
    )

    chain = in_context_prompt | llm
    return chain.invoke({"task_description": task_description, "examples": example_text, "input_text": input_text}).content

task_desc = "Преобразуй данный текст."
examples = [
    {"input": "hello", "output": "ellohay"},
    {"input": "apple", "output": "appleay"}
]
test_input = "python"

result = in_context_learning(task_desc, examples, test_input)
print(f"Ввод: {test_input}")
print(f"Вывод: {result}")

## Цепочка размышлений

Промтинг с помощью цепочки размышлений (Chain-of-Thoughts, CoT), представленный в работе [Wei et al. (2022)](https://arxiv.org/abs/2201.11903), позволяет LLM выполнять сложные задачи, требующие промежуточных шагов рассуждения. На популярном [бенчмарке](https://habr.com/ru/articles/840530/) по школьной арифметике GSM8K данный метод улучшает результат вдвое.

<center><img src="https://i.postimg.cc/J0q1n7LZ/CoT.png" width="800"></center>

In [ ]:
# Стандартный промпт
standard_prompt = PromptTemplate(
    input_variables=["question"],
    template="Кратко ответь на следующий вопрос: {question}."
)

# Промпт с цепочкой рассуждений
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="Кратко ответь на следующий вопрос шаг за шагом: {question}"
)

standard_chain = standard_prompt | llm
cot_chain = cot_prompt | llm

# Пример вопроса
question = "Прямоугольник имеет длину 10 см и ширину 5 см. Какова его площадь в квадратных сантиметрах?"

standard_response = standard_chain.invoke(question).content
cot_response = cot_chain.invoke(question).content

print("Стандартный ответ:")
print(standard_response)
print("\nОтвет CoT: ")
print(cot_response)


In [ ]:
advanced_cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="""Реши следующую задачу шаг за шагом. Для каждого шага:
1. Укажи, что ты собираешься рассчитать
2. Напиши формулу, которую будешь использовать (если применимо).
3. Выполни расчет
4. Объясни результат

Вопрос: {question}

Решение:"""
)

advanced_cot_chain = advanced_cot_prompt | llm

complex_question = "Цилиндрический резервуар имеет радиус 5 метров и высоту 10 метров. Сколько воды в кубических метрах он может вместить, если наполнить его на 80%?"

advanced_cot_response = advanced_cot_chain.invoke(complex_question).content
print(advanced_cot_response)

## Самосогласованность

Одной из продвинутых техник для создания промтов является самосогласованность (self-consistency). Эта техника была предложена в работе [Wang et al. (2022)](https://arxiv.org/abs/2203.11171) в качестве замены «жадного» декодирования, используемого в цепочках рассуждений (CoT). Идея заключается в том, чтобы сэмплировать несколько разнообразных путей рассуждений через Few-shot CoT и использовать эти генерации для выбора наиболее согласованного ответа. Это помогает улучшить производительность CoT-промтов в задачах, связанных с арифметическими и логическими рассуждениями.

Вначале сгенерируем несколько разных цепочек размышлений.

In [ ]:
def generate_multiple_paths(problem, num_paths=3):
  prompt_template = PromptTemplate(
      input_variables=["problem" , "path_number"],
      template="""Решите следующую задачу, каждый раз используя уникальный подход. Это способ рассуждения номер {path_number}.
      Задача: {problem}
      Способ рассуждения {path_number}:"""
  )
  paths = []
  for i in range(num_paths):
    chain = prompt_template | llm
    response = chain.invoke({"problem": problem, "path_number": i+1}).content
    paths.append(response)
  return paths

In [ ]:
problem = "Если поезд движется со скоростью 60 км/ч, сколько времени потребуется, чтобы преодолеть 180 км?"
paths = generate_multiple_paths(problem)

for i, path in enumerate(paths, 1):
  print(f"Способ {i}: \n{path}\n")

 Следующим шагом самосогласованности будет агрегация и анализ ответов, направленный на выявление самого оптимального, который и станет окончательным.

In [ ]:
def aggregate_results(paths):
    prompt_template = PromptTemplate(
        input_variables=["paths"],
        template="""Проанализируйте приведенные ниже рассуждения и найдите наиболее логичный ответ. Если есть расхождения, объясните, почему, и укажите наиболее вероятный правильный ответ.
        Способы рассуждения:
        {paths}

        Наиболее последовательный ответ:"""
    )

    chain = prompt_template | llm
    response = chain.invoke({"paths": "\n".join(paths)}).content
    return response

In [ ]:
aggregated_results = aggregate_results(paths)
print("Итоговый результат: \n", aggregated_results)

Можно проводить сравнение ответов по конкретным заданным критериям.

In [ ]:
def self_consistency_check(problem, aggregated_result):
    prompt_template = PromptTemplate(
        input_variables=["problem", "result"],
        template="""Оцените согласованность и надежность следующего результата для данной задачи.
        Задача: {problem}
        Результат: {result}

        Оценка (учитывай такие факторы, как логическая последовательность, соответствие известным фактам и потенциальные предубеждения):"""
    )

    chain = prompt_template | llm
    response = chain.invoke({"problem": problem, "result": aggregated_result}).content
    return response

In [ ]:
consistency_evaluation = self_consistency_check(problem, aggregate_results)
print("Оценка с помощью самосогласованности: \n", consistency_evaluation)

Аналогично можно решать различные задачи.

In [ ]:
def solve_problem(problem):
    paths = generate_multiple_paths(problem)
    aggregated_result = aggregate_results(paths)
    consistency_evaluation = self_consistency_check(problem, aggregated_result)
    return aggregated_result, consistency_evaluation

# Примеры задач
problems = [
    "Какой город является столицей Франции?",
    "Объясни концепцию спроса и предложения в экономике.",
    "Если поезд движется со скоростью 70 км/ч, сколько времени потребуется, чтобы преодолеть 180 км?"
]

for problem in problems:
    print(f"Задача: {problem}")
    result, evaluation = solve_problem(problem)
    print("Итоговый результат:\n", result)
    print("\nОценка согласованности:\n", evaluation)
    print("\n" + "-"*50 + "\n")

## Установка роли

Промпт может содержать указание, поведение какого специалиста должна имитировать LLM.

In [ ]:
tech_writer_prompt = PromptTemplate(
    input_variables=["topic"],
    template = """Ты технический писатель, специализирующийся на создании понятной и сжатой документации к программному продукту.
    Твоя задача — написать краткое объяснение темы {topic} для руководства пользователя.
    Пожалуйста, предоставь объяснение в 2-3 предложениях, которое будет легко понятно пользователям, не имеющим технических знаний"""
)
chain = tech_writer_prompt | llm
response = chain.invoke({"topic": "Машинное обучение"})
print(response.content)

In [ ]:
financial_advisor_prompt = PromptTemplate(
    input_variables=["client_situation"],
    template="""Вы опытный финансовый консультант с более чем 20-летним опытом работы в области личных финансов, инвестиционных стратегий и пенсионного планирования.
    У вас есть опыт оказания помощи клиентам из разных слоев общества в достижении их финансовых целей.
    Ваш подход характеризуется следующими принципами:
    1. Тщательный анализ уникальной финансовой ситуации каждого клиента
    2. Четкое изложение сложных финансовых концепций без использования жаргона
    3. Соблюдение этических норм во всех рекомендациях
    4. Акцент на долгосрочном финансовом благополучии и стабильности

    Учитывая следующую ситуацию с клиентом, предоставьте краткую (3-4 предложения) финансовую консультацию:
    {client_situation}

    Ваш ответ должен отражать ваш опыт и соответствовать вашему характерному подходу."""
)

chain = financial_advisor_prompt | llm
response = chain.invoke({"client_situation": "35-летний специалист, зарабатывающий 800 000 рублей в год, имеющий сбережения в размере 300 000 рублей, без долгов и пенсионного плана."})
print(response.content)

Объяснение одних и тех же понятий может отличаться для разных ролей.

In [ ]:
roles = [
    ("Ученый", "Вы ученый-исследователь, специализирующийся на изменении климата. Объясните следующую концепцию в научных терминах:"),
    ("Учитель", "Вы учитель естествознания в средней школе. Объясните следующую концепцию простыми словами, подходящими для 12-летних учеников:"),
    ("Журналист", "Вы журналист, пишущий для научно-популярного журнала. Объясните следующую концепцию в увлекательной и информативной форме для широкой взрослой аудитории:")
]

topic = "парниковый эффект"

for role, description in roles:
    role_prompt = PromptTemplate(
        input_variables=["topic"],
        template=f"{description} {{topic}}"
    )
    chain = role_prompt | llm
    response = chain.invoke({"topic": topic})
    print(f"\nОбъясняет {role}:\n")
    print(response.content)
    print("-" * 50)

In [ ]:
storyteller_prompt = PromptTemplate(
    input_variables=["style", "scenario"],
    template="""Вы прекрасный рассказчик, известный своей способностью адаптироваться к различным стилям повествования.
    Ваша текущая задача - писать в стиле {style}.
    Ключевые характеристики этого стиля включают:
    1. {style_char1}
    2. {style_char2}
    3. {style_char3}

    Напишите короткий абзац (3-4 предложения) в таком стиле о следующем сценарии:
    {scenario}

    Убедитесь, что ваш текст четко соответствует указанному стилю."""
)

styles = [
    {
        "name": "Готический хоррор",
        "char1": "Атмосферные и зловещие описания",
        "char2": "Темы разложения, смерти и сверхъестественного",
        "char3": "Обостренные эмоции и чувство страха"
    },
    {
        "name": "Минималистский реализм",
        "char1": "Скупой, лаконичный язык",
        "char2": "Фокус на повседневных, заурядных событиях",
        "char3": "Тонкие намеки, а не явные заявления"
    }
]

scenario = "Человек входит в пустой дом в сумерках"

for style in styles:
    chain = storyteller_prompt | llm
    response = chain.invoke({
        "style": style["name"],
        "style_char1": style["char1"],
        "style_char2": style["char2"],
        "style_char3": style["char3"],
        "scenario": scenario
    })
    print(f"\n{style['name']}:\n")
    print(response.content)
    print("-" * 50)